# [Create Agent With OpenAPI](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview#create-agent-with-openapi)
AzureFunctionTool contains the input and output queues of azure function and the description of input parameters.

Inspired by [sample_agents_openapi.py](https://github.com/Azure/azure-sdk-for-python/blob/azure-ai-projects_1.0.0b4/sdk/ai/azure-ai-projects/samples/agents/sample_agents_openapi.py)

# Constants

In [ ]:
import os, jsonref
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4o-0513" # https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview#setup
project_connection_string = os.environ["PROJECT_CONNECTION_STRING"]

print(f'Project Connection String: <...{project_connection_string[-30:]}>')

In [ ]:
# Create AI Foundry Project Client

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails # <<<<<<<<<<<<<<< SPECIFIC FOR OPENAPI
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

project_client.scope

In [ ]:
# Load Weather OpenAPI

In [ ]:
# https://github.com/Azure/azure-sdk-for-python/blob/azure-ai-projects_1.0.0b4/sdk/ai/azure-ai-projects/samples/agents/weather_openapi.json
with open("./openapi/weather_openapi.json", "r") as f:
    openapi_spec = jsonref.loads(f.read())

openapi_spec

# Create Auth object for the OpenApiTool 
## Note: connection or managed identity auth setup requires additional setup in Azure

In [ ]:
auth = OpenApiAnonymousAuthDetails()
auth

# Initialize agent OpenApi tool using the read in OpenAPI spec

In [ ]:
openapi = OpenApiTool(
    name="get_weather", spec=openapi_spec, description="Retrieve weather information for a location", auth=auth
)
print(f"openapi.definitions: {openapi.definitions}")
print(f"openapi.resources: {openapi.resources}")

# Create AI Foundry Agent

In [ ]:
# Create agent with AI search tool and process assistant run
agent = project_client.agents.create_agent(
    model=model_name,
    name="openapi-agent",
    instructions="You are a helpful assistant",
    tools=openapi.definitions
)

agent.items

# Create the thread and attach a new message to it

In [ ]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id, 
    role="user", 
    content="Quale sarà la temperatura massima a Chiavari il 15 giugno? Se il giorno non è ancora disponibile, indica esattamente quale è l'ultima data per la quale sei in grado di rispondere.",
)
print(f"Created message: {message}")

# Run the agent synchronously

In [ ]:
%%time
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}. Run: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

# Fetch messages from the thread after the agent run execution

In [ ]:
# Fetch messages from the thread
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}\n")

# Fetch the last message from the thread
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message:\n{last_msg.text.value}")

In [ ]:
# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

# Run Steps

In [ ]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)

print(f'Nr of run step(s): {len(run_steps["data"])}\n')
i=0
for rs in run_steps["data"]:
    i += 1
    print(f"Run step {i}: {rs}", '\n')

# START teardown

In [ ]:
print(f"deleting trhead: {thread}...")
project_client.agents.delete_thread(thread.id)

In [ ]:
# Delete all agents

print(f"{len(project_client.agents.list_agents()['data'])} agent(s) will now be deleted")

i=0
for pca in project_client.agents.list_agents()['data']:
    i += 1
    project_client.agents.delete_agent(pca.id)
    print(f"\n{i} - Agent {pca.name} has been deleted")

# HIC SUNT LEONES